In [ ]:
rom torchvision import models
from torchsummary import summary
import torch
import torch.nn as nn
from torchvision import utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
import os
os.mkdir('./train_cifar10')
os.mkdir('./test_cifar10')

In [ ]:
transform = transforms.ToTensor()
train_dataset = datasets.CIFAR10(
    root='./train_cifar10', train=True, transform=transform)
test_dataset = datasets.CIFAR10(
    root='./test_cifar10', train=False, transform=transform)


In [ ]:
!pip install torchsummary

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet34(pretrained=False).to(device)

summary(model, (3, 32, 32))

In [ ]:
num_features = model.fc.in_features
model.fc.in_features = nn.Linear(num_features, 10)
model.conv1.kernel_size = 3
model.conv1.padding = 1